## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:

![figure](./fig/hw4-q1.jpg)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：

In [1]:
# add your code here 过滤所有权重大于b的边，生成子图G'。检查G'是否连通。若连通，则存在BST的最大边不超过b。
from collections import defaultdict

def has_bottleneck(vertices, edges, b):
    adj = defaultdict(list)
    for u, v, w in edges:
        if w <= b:
            adj[u].append(v)
            adj[v].append(u)
    
    if not vertices:
        return True
    visited = set()
    start = next(iter(vertices))
    stack = [start]
    visited.add(start)
    while stack:
        u = stack.pop()
        for v in adj[u]:
            if v not in visited:
                visited.add(v)
                stack.append(v)
    return len(visited) == len(vertices)

# algorithm of the liear time complexity O(V+E)



# Q3 二分查找确定最小b 时间复杂度O(V + E)
def find_bottleneck_tree(vertices, edges):
    if not edges:
        return [] if len(vertices) <= 1 else None
    
    weights = [w for _, _, w in edges]
    left, right = min(weights), max(weights)
    b = right  # 初始化为最大边权
    
    # 二分查找最小b
    while left <= right:
        mid = (left + right) // 2
        if has_bottleneck(vertices, edges, mid):
            b = mid
            right = mid - 1
        else:
            left = mid + 1
    
    # 构造生成树
    filtered_edges = [(u, v, w) for u, v, w in edges if w <= b]
    parent = {v: v for v in vertices}
    
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    
    tree = []
    for u, v, w in filtered_edges:
        root_u, root_v = find(u), find(v)
        if root_u != root_v:
            tree.append((u, v, w))
            parent[root_v] = root_u
        if len(tree) == len(vertices) - 1:
            break
    
    return tree if len(tree) == len(vertices) - 1 else None

## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [2]:
#借助deepseek完成
import heapq

def dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    heap = [(0, start)]
    
    while heap:
        current_dist, current_node = heapq.heappop(heap)
        
        if current_dist > dist[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            distance = current_dist + weight
            if distance < dist[neighbor]:
                dist[neighbor] = distance
                heapq.heappush(heap, (distance, neighbor))
    
    return dist

def find_best_new_edge(G, E_prime, s, t):
    # 计算 s 到所有节点的最短路径
    d_s = dijkstra(G, s)
    
    # 计算所有节点到 t 的最短路径
    d_t = dijkstra(G, t)
    
    # 计算当前 s 到 t 的最短路径
    d_old = d_s[t]
    
    max_reduction = 0
    best_edge = None
    
    # 遍历所有候选边
    for (u, v), l_prime in E_prime.items():
        # 新路径长度：d_s[u] + l'(u, v) + d_t[v]
        new_path_length = d_s[u] + l_prime + d_t[v]
        
        # 如果新路径比原路径短，计算减少的距离
        reduction = d_old - new_path_length
        if reduction > max_reduction:
            max_reduction = reduction
            best_edge = (u, v)
    
    return best_edge, max_reduction

# 示例用法
if __name__ == "__main__":
    # 当前图 G
    G = {
        'A': {'B': 5, 'C': 3},
        'B': {'D': 2},
        'C': {'D': 4},
        'D': {}
    }
    
    # 候选新边 E'
    E_prime = {
        ('A', 'D'): 1,
        ('B', 'C'): 6,
        ('C', 'B'): 7
    }
    
    # 起点 s 和终点 t
    s = 'A'
    t = 'D'
    
    # 找到最优的新边
    best_edge, reduction = find_best_new_edge(G, E_prime, s, t)
    print(f"最优的新边: {best_edge}, 减少的距离: {reduction}")

最优的新边: ('A', 'D'), 减少的距离: 6


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：

In [3]:
from collections import deque

# 网格大小
n = 5
m = 3

# 起点坐标
start_points = [(1, 1), (2, 2), (3, 3)]

# 边界顶点集合
boundary_points = set()
for i in range(1, n + 1):
    boundary_points.add((i, 1))  # 左边界
    boundary_points.add((i, n))  # 右边界
    boundary_points.add((1, i))  # 上边界
    boundary_points.add((n, i))  # 下边界

# 网格顶点编号
def get_vertex_id(i, j):
    return (i - 1) * n + j - 1

# 构建网络流图
def build_flow_network(n, start_points, boundary_points):
    # 顶点总数：网格顶点 + 源点 + 汇点
    num_vertices = n * n + 2
    source = n * n
    sink = n * n + 1
    
    # 邻接表表示的网络
    graph = [[] for _ in range(num_vertices)]
    
    # 添加源点到起点的边
    for x, y in start_points:
        u = get_vertex_id(x, y)
        graph[source].append((u, 1))
        graph[u].append((source, 0))
    
    # 添加边界点到汇点的边
    for x, y in boundary_points:
        v = get_vertex_id(x, y)
        graph[v].append((sink, 1))
        graph[sink].append((v, 0))
    
    # 添加网格内部的边
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            u = get_vertex_id(i, j)
            if i < n:  # 下方邻居
                v = get_vertex_id(i + 1, j)
                graph[u].append((v, 1))
                graph[v].append((u, 0))
            if j < n:  # 右方邻居
                v = get_vertex_id(i, j + 1)
                graph[u].append((v, 1))
                graph[v].append((u, 0))
    
    return graph, source, sink

# Edmonds-Karp 算法求最大流
def edmonds_karp(graph, source, sink):
    n = len(graph)
    max_flow = 0
    
    while True:
        # BFS 找增广路径
        parent = [-1] * n
        queue = deque([source])
        parent[source] = source
        
        while queue:
            u = queue.popleft()
            for v, capacity in graph[u]:
                if parent[v] == -1 and capacity > 0:
                    parent[v] = u
                    if v == sink:
                        break
                    queue.append(v)
        
        # 如果找不到增广路径，结束
        if parent[sink] == -1:
            break
        
        # 更新残余网络
        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            for idx, (to, cap) in enumerate(graph[u]):
                if to == v:
                    path_flow = min(path_flow, cap)
                    break
            v = u
        
        v = sink
        while v != source:
            u = parent[v]
            for idx, (to, cap) in enumerate(graph[u]):
                if to == v:
                    graph[u][idx] = (to, cap - path_flow)
                    break
            for idx, (to, cap) in enumerate(graph[v]):
                if to == u:
                    graph[v][idx] = (to, cap + path_flow)
                    break
            v = u
        
        max_flow += path_flow
    
    return max_flow

# 主函数
def solve_escape_problem(n, start_points, boundary_points):
    graph, source, sink = build_flow_network(n, start_points, boundary_points)
    max_flow = edmonds_karp(graph, source, sink)
    return max_flow

# 示例运行
max_flow = solve_escape_problem(n, start_points, boundary_points)
print("最大流:", max_flow)

最大流: 3
